In [1]:
import torch
import numpy as np
from PIL import Image
import pdb
import os
import cv2
from scipy.io import loadmat,savemat
from collections import defaultdict
import tqdm

In [39]:
data_path = "/home/zhengwei/Desktop/Zhengwei/Projects/datasets/Market-Sketch-1K"
train_style = ['A', 'B', 'C', 'D', 'E', 'F']
files_rgb = os.listdir(data_path+'/photo/train')
files_sk = {s: os.listdir(f'{data_path}/sketch/{s}/train') for s in train_style}

In [40]:
pid_container = set()
for s in files_sk.keys():
    files = files_sk[s]
    for img_path in files:
        pid = int(img_path[:4])
        pid_container.add(pid)
pid2label = {pid:label for label, pid in enumerate(pid_container)}


_train_image = defaultdict(list)
_train_sketch = defaultdict(list)

sketch2idx = {s: i for i, s in enumerate(train_style)}

for s in files_sk.keys():
    train_image = files_sk[s]
    for img_path in train_image:
        pid = int(img_path[:4])
        _train_image[pid].append(f'sketch/{s}/train/{img_path}')
        _train_sketch[pid].append(sketch2idx[s])


In [41]:
print(_train_sketch.keys())
# count the min length of images list in each id
min_len = 100000
for key in _train_image.keys():
    min_len = min(min_len, len(_train_image[key]))
print(min_len)


dict_keys([818, 936, 466, 757, 352, 321, 122, 480, 528, 11, 385, 593, 566, 211, 806, 423, 379, 926, 707, 957, 779, 718, 555, 407, 930, 166, 175, 828, 902, 832, 674, 392, 209, 890, 496, 331, 648, 434, 42, 599, 780, 882, 482, 177, 398, 151, 149, 900, 658, 134, 760, 886, 224, 339, 519, 933, 656, 537, 947, 197, 676, 46, 744, 752, 948, 374, 266, 540, 206, 450, 850, 522, 649, 895, 810, 259, 642, 941, 81, 181, 444, 594, 703, 414, 677, 584, 481, 359, 701, 563, 891, 714, 659, 613, 821, 697, 296, 961, 782, 317, 653, 472, 793, 552, 939, 446, 110, 464, 371, 59, 30, 357, 76, 108, 809, 115, 90, 236, 513, 180, 572, 615, 79, 349, 640, 893, 167, 855, 141, 761, 93, 755, 347, 796, 759, 245, 833, 823, 554, 545, 281, 408, 250, 683, 27, 237, 958, 53, 588, 570, 28, 397, 344, 611, 243, 225, 410, 662, 348, 525, 449, 248, 376, 517, 508, 327, 534, 843, 766, 565, 354, 173, 647, 201, 633, 82, 539, 143, 159, 484, 411, 663, 249, 702, 575, 43, 390, 148, 216, 872, 121, 603, 622, 195, 597, 314, 742, 299, 370, 214, 69, 

In [36]:
import shutil

def copy_file(src_path, dst_path):
    shutil.copy(src_path, dst_path)

In [37]:
# seed manual seed for random function
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

few_shot_basepath = '/home/zhengwei/Desktop/Zhengwei/Projects/datasets/Market-Sketch-1K/sketch/fewshot'

for pid in tqdm.tqdm(sorted(_train_image.keys())):
    if not pid in pid2label.keys():
            continue
    img_paths = _train_image[pid]
    sketch_idx = _train_sketch[pid]

    # print(img_paths)

    # # select a random number from 0 to len(sketch_idx)
    # select = np.random.randint(0, len(sketch_idx))

    # select 2 random numbers from 0 to len(sketch_idx)
    select = np.random.choice(len(sketch_idx), 2, replace=False)
    # selected_sketch = sketch_idx[select]
    # unselected_sketch = [sketch_idx[i] for i in range(len(sketch_idx)) if i != select]

    # print(sketch_idx)
    # print(selected_sketch)
    # print(unselected_sketch)
    
    for idx, img_path in enumerate(img_paths):
        style = train_style[sketch_idx[idx]]
        img = img_path.split('/')[-1][:4]
        if idx in select:
            # copy image_file from one path to another, with out open it
            copy_file(f'{data_path}/{img_path}', f'{few_shot_basepath}/2sketch/finetune/{img}_{style}.jpg')
        else:
            copy_file(f'{data_path}/{img_path}', f'{few_shot_basepath}/2sketch/test/{img}_{style}.jpg')
    # break




100%|██████████| 498/498 [00:00<00:00, 775.63it/s]


In [12]:
import os

def print_directory_structure(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for idx, f in enumerate(sorted(files)):
            if idx < 6:
                print('{}{}'.format(subindent, f))

# Replace 'your_directory_path' with the path of the directory you want to print
print_directory_structure(few_shot_basepath)

fewshot/
    train/
        finetune/
            0002_E.jpg
            0007_F.jpg
            0010_A.jpg
            0011_D.jpg
            0012_D.jpg
            0020_D.jpg
        test/
            0002_A.jpg
            0002_B.jpg
            0002_C.jpg
            0002_D.jpg
            0002_F.jpg
            0007_A.jpg
    all/
        finetune/
            0001_F.jpg
            0002_E.jpg
            0003_A.jpg
            0004_E.jpg
            0005_E.jpg
            0006_E.jpg
        test/
            0001_A.jpg
            0001_C.jpg
            0001_D.jpg
            0001_E.jpg
            0002_A.jpg
            0002_B.jpg


In [9]:
import os
import pandas as pd
from tabulate import tabulate

def file_structure(path):
    file_dict = {}
    for root, dirs, files in os.walk(path):
        # Use the relative path as the header, with 'Base' indicating the root directory
        relative_path = os.path.relpath(root, path) if os.path.relpath(root, path) != '.' else 'Base'
        file_dict[relative_path] = len(files)
    return file_dict

file_dict = file_structure(few_shot_basepath)

# Creating a DataFrame from the dictionary, with directory structures as columns
df = pd.DataFrame([file_dict])

# Optional: If you want to sort the columns to reflect the directory structure hierarchy
# df = df.sort_index(axis=1)

# Printing the table using tabulate
print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))


+--------+---------+------------------+--------------+-------+----------------+------------+
|   Base |   train |   train/finetune |   train/test |   all |   all/finetune |   all/test |
|--------+---------+------------------+--------------+-------+----------------+------------|
|      0 |       0 |              498 |         1833 |     0 |            996 |       3705 |
+--------+---------+------------------+--------------+-------+----------------+------------+


In [42]:
# according to the files in /home/zhengwei/Desktop/Zhengwei/Projects/datasets/Market-Sketch-1K/sketch/fewshot/2sketch
# adjust copy file from  /home/zhengwei/Desktop/Zhengwei/Projects/datasets/Market-Sketch-1K/tensor/CLIPreidNew/sketch/fewshot/all to  /home/zhengwei/Desktop/Zhengwei/Projects/datasets/Market-Sketch-1K/tensor/CLIPreidNew/sketch/fewshot/2sketch

